In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk import corpus
import re
import ast
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import datasets, linear_model

## EDA of Recipe & Interactions

Loading the recipe and interactions data.

In [2]:
Raw_recipes = pd.read_csv('Capstone/raw_recipes.csv')
Raw_ints = pd.read_csv('Capstone/raw_interactions.csv')

In [3]:
df_r = Raw_recipes.drop_duplicates('name')
df_r.shape

(230186, 12)

In [4]:
df_r = df_r.dropna()
df_r.shape

(225248, 12)

In [5]:
df_r2 = df_r.reset_index(drop=True)

In [6]:
df_r2.head(5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']","autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions.","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment']","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough into the bottom and sides of a 12 inch pizza pan', 'bake for 5 minutes until set but not browned', 'cut sausage into small pieces', 'whisk eggs and milk in a bowl until frothy', 'spoon sausage over baked crust and sprinkle with cheese', 'pour egg mixture slowly over sausage and cheese', 's& p to taste', 'bake 15-20 minutes or until eggs are set and crust is brown']",this recipe calls for the crust to be prebaked a bit before adding ingredients. feel free to change sausage to ham or bacon. this warms well in the microwave for those late risers.,"['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add chopped onions to ground beef when almost brown and sautee until wilted', 'add all other ingredients', 'add kidney beans if you like beans in your chili', 'cook in slow cooker on high for 2-3 hours or 6-8 hours on low', 'serve with cold clean lettuce and shredded cheese']",this modified version of 'mom's' chili was a hit at our 2004 christmas party. we made an extra large pot to have some left to freeze but it never made it to the freezer. it was a favorite by all. perfect for any cold and rainy day. you won't find this one in a cookbook. it is truly an original.,"['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'potatoes', 'vegetables', 'oven', 'easy', 'dinner-party', 'holiday-event', 'easter', 'cheese', 'stove-top', 'dietary', 'christmas', 'new-years', 'thanksgiving', 'independence-day', 'st-pat

Extracting Nutrition column, and giving their respective column names and values per Recipe.

In [7]:
Nutri_Cols = pd.DataFrame(df_r2['nutrition'].str.strip('[]').str.replace(',', '').str.split().to_list())

Giving the column names to each nutrition value point.

In [8]:
Nutri_Cols.columns = ['Calories (#)', 'Total Fat (PDV)', 'Sugar (PDV)', 'Sodium (PDV)', 'Protein (PDV)', 'Saturated Fat', 'Carbohydrates (PDV)'] 


Here's the Nutrition column and it's respective values per recipe.

In [9]:
Nutri_Cols.head(5)

,Calories (#),Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat,Carbohydrates (PDV)
0,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,352.9,1.0,337.0,23.0,3.0,0.0,28.0


Let's take this table and add it back to the cleaned recipe data table.

In [10]:
c = Raw_recipes.drop(columns = ['nutrition'])
b = c.join(Nutri_Cols)


Let's now combine the user rating with the recipes from the interaction table.

In [11]:
b.head(5)

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,Calories (#),Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat,Carbohydrates (PDV)
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",11,"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']","autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions.","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment']",9,"['preheat oven to 425 degrees f', 'press dough into the bottom and sides of a 12 inch pizza pan', 'bake for 5 minutes until set but not browned', 'cut sausage into small pieces', 'whisk eggs and milk in a bowl until frothy', 'spoon sausage over baked crust and sprinkle with cheese', 'pour egg mixture slowly over sausage and cheese', 's& p to taste', 'bake 15-20 minutes or until eggs are set and crust is brown']",this recipe calls for the crust to be prebaked a bit before adding ingredients. feel free to change sausage to ham or bacon. this warms well in the microwave for those late risers.,"['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']",6,"['brown ground beef in large pot', 'add chopped onions to ground beef when almost brown and sautee until wilted', 'add all other ingredients', 'add kidney beans if you like beans in your chili', 'cook in slow cooker on high for 2-3 hours or 6-8 hours on low', 'serve with cold clean lettuce and shredded cheese']",this modified version of 'mom's' chili was a hit at our 2004 christmas party. we made an extra large pot to have some left to freeze but it never made it to the freezer. it was a favorite by all. perfect for any cold and rainy day. you won't find this one in a cookbook. it is truly an original.,"['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'potatoes', 'vegetables', 'oven', 'easy', 'dinner-party', 'holiday-event', 'easter', 'cheese', 'stove-top', 'dietary', 'christ

Let's run the same excercise for User Interactions Data and clean out the data for duplicates as well as na's

In [12]:
Raw_ints.head(5)
Raw_ints.shape

(1132367, 5)

Droping duplicates from the review column.

In [13]:
df_int = Raw_ints.drop_duplicates('review')
df_int.shape


(1125283, 5)

Dataset has more than 7000 duplicates in review column now removed.

Now we are dropping any Na's in the dataset.

In [14]:
df_int = df_int.dropna()
df_int.shape

(1125282, 5)

In [15]:
df_int2 = df_int.reset_index(drop=True)

Final data frame will show the most cleaned data table for User Interactions.

In [16]:
df_int2.head(5)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for 15 minutes.Added a shake of cayenne and a pinch of salt. Used low fat sour cream. Thanks.
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall evening. Should have doubled it ;)<br/><br/>Second time around, forgot the remaining cumin. We usually love cumin, but didn't notice the missing 1/2 teaspoon!"
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not quite a whole package (10oz) of white chips. Great!
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunko. Everyone loved it.
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprinkling of black pepper. Yum!"


In [17]:
df_int2.columns = ['user', 'id', 'date', 'rate', 'rev']

In [18]:
resultdf =b.merge(df_int2,how="inner",on="id")

In [19]:
resultdf.head(5)

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,...,Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat,Carbohydrates (PDV),user,date,rate,rev
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",11,"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']","autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions.","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",...,0.0,13.0,0.0,2.0,0.0,4.0,4470,2006-02-18,5,"I used an acorn squash and recipe#137681 Sweet Mexican spice blend. Only used 1 tsp honey & 1 tsp butter between both halves,, sprinkled the squash liberally with the spice mix. Baked covered for 45 minutes uncovered or 15. I basted the squash with the the butter/honey from the cavity allowing it to get a golden color. Lovely Squash recipe Thanks Cookgirl"
1,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",11,"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']","autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions.","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",...,0.0,13.0,0.0,2.0,0.0,4.0,593927,2010-08-21,5,This was a nice change. I used butternut squash and the sweet option using a good local honey and unsalted butter. I did not add salt. We ate this on top of recipe#322603 with Balkan yogurt. I may make this again same option. Made for Ramadan Tag 2010.
2,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",11,"['mak

In [20]:
resultdf.shape

(1125282, 22)

Check for duplicates on the master table.

In [21]:
bool_series = resultdf.duplicated()

In [22]:
bool_series

0          False
1          False
2          False
3          False
4          False
           ...  
1125277    False
1125278    False
1125279    False
1125280    False
1125281    False
Length: 1125282, dtype: bool

This verifies that all 1.1 million user interactive columns are unique and not duplicates.

In [23]:
# Applying the method
check_nan = resultdf.isnull().values.any()

In [24]:
check_nan

True

In [25]:
# applying the method
count_nan = resultdf.isnull().sum()
 
# printing the number of values present
# in the column
print('Number of NaN values present: ' + str(count_nan))

Number of NaN values present: name                       1
id                         0
minutes                    0
contributor_id             0
submitted                  0
tags                       0
n_steps                    0
steps                      0
description            23334
ingredients                0
n_ingredients              0
Calories (#)           33206
Total Fat (PDV)        33206
Sugar (PDV)            33206
Sodium (PDV)           33206
Protein (PDV)          33206
Saturated Fat          33206
Carbohydrates (PDV)    33206
user                       0
date                       0
rate                       0
rev                        0
dtype: int64


In [26]:
df2 = resultdf.dropna()

In [27]:
df2.shape

(1069273, 22)

56000 rows were removed after the 'dropna'

In [28]:
# Applying the method
check_nan = df2.isnull().values.any()

In [29]:
check_nan

False

This validates that the data is clean, no nan's or missing values.

## REGEX CLEAN VALUES

We will clean the string values that would be required for further use in the recommendation system.
Ingredients is great.
Steps is another great option.
Tags are also amazing to do this with as well.

In [30]:
df2['ingredients'] = df2['ingredients'].str.replace('[^\w\s]','')  

/var/folders/3h/c9q_p3653g7088wkgdk7bfmm0000gn/T/ipykernel_6092/573432464.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['ingredients'] = df2['ingredients'].str.replace('[^\w\s]','')
/var/folders/3h/c9q_p3653g7088wkgdk7bfmm0000gn/T/ipykernel_6092/573432464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ingredients'] = df2['ingredients'].str.replace('[^\w\s]','')


In [31]:
df2['steps'] = df2['steps'].str.replace('[^\w\s]','')  

/var/folders/3h/c9q_p3653g7088wkgdk7bfmm0000gn/T/ipykernel_6092/196190552.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['steps'] = df2['steps'].str.replace('[^\w\s]','')
/var/folders/3h/c9q_p3653g7088wkgdk7bfmm0000gn/T/ipykernel_6092/196190552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['steps'] = df2['steps'].str.replace('[^\w\s]','')


In [32]:
df2['tags'] = df2['tags'].str.replace('[^\w\s]','')  

/var/folders/3h/c9q_p3653g7088wkgdk7bfmm0000gn/T/ipykernel_6092/918353815.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['tags'] = df2['tags'].str.replace('[^\w\s]','')
/var/folders/3h/c9q_p3653g7088wkgdk7bfmm0000gn/T/ipykernel_6092/918353815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['tags'] = df2['tags'].str.replace('[^\w\s]','')


In [33]:
df2_clean = df2.copy()
df2_clean.shape

(1069273, 22)

Now the final dataset sits at 1.06 million rows, combining user review and recipe details in 1 table.

In [34]:
df2_clean.head(5)

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,...,Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat,Carbohydrates (PDV),user,date,rate,rev
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,60minutesorless timetomake course mainingredient cuisine preparation occasion northamerican sidedishes vegetables mexican easy fall holidayevent vegetarian winter dietary christmas seasonal squash,11,make a choice and proceed with recipe depending on size of squash cut into half or fourths remove seeds for spicy squash drizzle olive oil or melted butter over each cut squash piece season with mexican seasoning mix ii for sweet squash drizzle melted honey butter grated piloncillo over each cut squash piece season with sweet mexican spice mix bake at 350 degrees again depending on size for 40 minutes up to an hour until a fork can easily pierce the skin be careful not to burn the squash especially if you opt to use sugar or butter if you feel more comfortable cover the squash with aluminum foil the first half hour give or take of baking if desired season with salt,"autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions.",winter squash mexican seasoning mixed spice honey butter olive oil salt,...,0.0,13.0,0.0,2.0,0.0,4.0,4470,2006-02-18,5,"I used an acorn squash and recipe#137681 Sweet Mexican spice blend. Only used 1 tsp honey & 1 tsp butter between both halves,, sprinkled the squash liberally with the spice mix. Baked covered for 45 minutes uncovered or 15. I basted the squash with the the butter/honey from the cavity allowing it to get a golden color. Lovely Squash recipe Thanks Cookgirl"
1,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,60minutesorless timetomake course mainingredient cuisine preparation occasion northamerican sidedishes vegetables mexican easy fall holidayevent vegetarian winter dietary christmas seasonal squash,11,make a choice and proceed with recipe depending on size of squash cut into half or fourths remove seeds for spicy squash drizzle olive oil or melted butter over each cut squash piece season with mexican seasoning mix ii for sweet squash drizzle melted honey butter grated piloncillo over each cut squash piece season with sweet mexican spice mix bake at 350 degrees again depending on size for 40 minutes up to an hour until a fork can easily pierce the skin be careful not to burn the squash especially if you opt to use sugar or butter if you feel more comfortable cover the squash with aluminum foil the first half hour give or take of baking if desired season with salt,"autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions.",winter squash mexican seasoning mixed spice honey butter olive oil salt,...,0.0,13.0,0.0,2.0,0.0,4.0,593927,2010-08-21,5,This was a nice change. I used butternut squash and the sweet option using a good local honey and unsalted butter. I did not add salt. We ate this on top of recipe#322603 with Balkan yogurt. I may make this again same option. Made for Ramadan Tag 2010.
2,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,60minutesorless timetomake course mainingredient cuisine preparation occasion northamerican sidedishes vegetables mexican easy fall holidayevent vegetarian winter dietary christmas seasonal squash,11,make a choice and proceed with recipe depending on size of squash cut into half or fourths remove seeds for spicy squash drizzle olive oil or melted butter over each cut squash piece season with mexican seasoning mix ii for sweet squash drizzle melted honey butter grated piloncillo over each cut squash piece season with sweet mexican spice mix bake at 350 degrees again depending on size 

Making a count for the users in the dataframe

In [35]:
df2_usercount = df2_clean.groupby(['user']).count().reset_index()

In [36]:
df2_usercount.head(5)

,user,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,...,Calories (#),Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat,Carbohydrates (PDV),date,rate,rev
0,1533,119,119,119,119,119,119,119,119,119,...,119,119,119,119,119,119,119,119,119,119
1,1535,738,738,738,738,738,738,738,738,738,...,738,738,738,738,738,738,738,738,738,738
2,1581,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1634,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56
4,1676,29,29,29,29,29,29,29,29,29,...,29,29,29,29,29,29,29,29,29,29


Seperating the column and then joining it back with the original 1.1 million dataset.

In [37]:
df2_usercounta = df2_usercount[['user','name']]
df2_usercounta.columns = ['user', 'count']
df2_usercounta.sort_values('count')
resultdf2 =df2_usercounta.merge(df2_clean,how="inner",on="user")
resultdf2.shape

(1069273, 23)

In [38]:
resultdf2.shape

(1069273, 23)

New dataframe now includes Vote count and rating value.  We can try content based recommendation

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words = "english", min_df=2)
resultdf2['description'] = resultdf2['description'].fillna("")

TF_IDF_matrix = vectorizer.fit_transform(resultdf2['description'])

In [40]:
TF_IDF_matrix.shape

(1069273, 53302)

In [41]:
TF_IDF_matrix[(resultdf2['name'] == 'pizza').values].todense().squeeze()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
resultdf2[resultdf2['name'].str.contains('pizza', na=False)]

,user,count,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,...,Calories (#),Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat,Carbohydrates (PDV),date,rate,rev
87,1533,119,pizza breadsticks,17387,15,4470,2002-01-18,15minutesorless timetomake course mainingredient cuisine preparation northamerican appetizers breads eggsdairy canadian oven easy cheese freezer dietary equipment numberofservings 3stepsorless,3,heat oven to 300f place bread sticks about an inch apart on a baking sheet foil lined and lightly sprayed with veggie oil in a bowl mix tomatoe paste water oil chili pepper mix well brush the tops of the bread sticks with this mixture sprinkle the sticks with the oregano basil and mozzarella cheese bake bread sticks until heated through 35 minutes and the cheese has melted cool 1 minute serve warm,...,831.8,11.0,41.0,136.0,63.0,8.0,58.0,2002-07-28,5,"I love bread stick and these were perfect, I did make ahead and froze them. They turned out great.\r\nAnother recipe that got mailed out today. I think this was the easiest recipe of them all, thanks Bergy for such great recipes ;-) oh I did triple the recipe."
88,1533,119,pizzatizers,27644,25,23302,2002-05-08,30minutesorless timetomake course mainingredient cuisine preparation occasion northamerican appetizers lunch eggsdairy fruit american canadian southernunitedstates oven easy beginnercook diabetic dinnerparty fingerfood holidayevent kidfriendly vegetarian grains cheese dietary spicy comfortfood pacificnorthwest inexpensive northeasternunitedstates pastariceandgrains tastemood equipment presentation,7,separate each pita into two spread 1 garlic clove onto each pita round arrange 46 basil leaves over garlic top with 3 slices of tomatoes sprinkle about 2 tablespoons of shredded cheese on each and drizzle 2 teaspoons of olive oil on each bake on cookie sheet in 400degree oven for about 15 minutes or until cheese melts and pizzas are bubbly serve immediately,...,652.8,63.0,25.0,40.0,34.0,56.0,17.0,2002-07-28,5,"These were wonderful, I cut back on the oil. I used 8 pitas and only 2 tsps. of oil. Left everything else the same and cut each into 4 for appetizers. Great recipe miller, fast easy, just the way I like it. A keeper."
154,1535,738,bagel pizza,94735,5,148154,2004-06-30,15minutesorless timetomake course preparation for1or2 5ingredientsorless lunch easy dietary highprotein highinsomething numberofservings,5,set oven to broil on high split bagel in half and place on cookie sheet top with ham first then pepperoni then cheese broil for about 2 or 3 minutes or until cheese is completely melted do not brown the cheese,...,489.5,70.0,0.0,39.0,35.0,75.0,0.0,2006-03-23,5,"Quick, easy and tasty. I jazzed it up just a bit with a sprinkling of Italian Seasonings on the top. Thanks for sharing!"
196,1535,738,black forest pizza,43814,45,20754,2002-10-22,60minutesorless timetomake course mainingredient cuisine preparation occasion northamerican desserts lunch fruit american oven easy potluck holidayevent kidfriendly chocolate dietary christmas valentinesday togo equipment numberofservings,11,preheat oven to 350 degrees coat a 12inch pizza pan with nonstick cooking spray prepare brownie mix according to package directions spread the prepared batter evenly over the bottom of the pan bake for 2025 minutes or until a wooden toothpick inserted in center comes out clean let cool in a 23 quart mixing bowl beat the cream cheese confectioners sugar and whipped topping until smooth spread the mixture evenly over the brownie crust spoon on the cherry pie filling spread to about 1 inch of the outer edge sprinkle chocolate candy sprinkles on the outer white rim chill and serve makes 12 slices,...,111.4,0.0,45.0,0.0,0.0,0.0,3.0,2006-04-03,5,"This turned out beautifully! I used my Rich, Fudgy Cocoa Brownies recipe #95357 for the brownie layer and then proceeded as directed. Very easy to put together but very impressive looking. I got a lot of oohs and ahhs when I uncovere

Cosine simlarity for content based will be used

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

recipe_n1 = TF_IDF_matrix[(resultdf2['name'] == 'cream cheese pizza dip').values,]
recipe_n2 = TF_IDF_matrix[(resultdf2['name'] == 'vegan pizza dough').values,]

print("Similarity:", cosine_similarity(recipe_n1, recipe_n2))

Similarity: [[0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696]
 [0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696]
 [0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696]
 [0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696 0.01252696 0.01252696 0.01252696
  0.01252696 0.01252696 0.01252696]
 [0.01252696 0.01252696 0.01

Now the most computationally heavy aspect is completing this similarities matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity 
similarities = cosine_similarity(TF_IDF_matrix, dense_output=False)

In [ ]:
similarities.shape

In [ ]:
resultdf2[resultdf2['name'] == 'cream cheese pizza dip']

In [ ]:
# Get the column based upon the index
movie_index = resultdf2[resultdf2['name'] == 'cream cheese pizza dip'].index

# Create a dataframe with the movie titles
sim_df = pd.DataFrame({'recipe':resultdf2['name'], 
                       'similarity': np.array(similarities[movie_index, :].todense()).squeeze()})

In [ ]:
# Return the top 10 most similar movies
sim_df.sort_values(by='similarity', ascending=False).head(10)

In [ ]:
def content_recommender(title, movies, similarities, vote_threshold=10) :
    
    # Get the movie by the title
    movie_index = movies[movies['title'] == title].index
    
    # Create a dataframe with the movie titles
    sim_df = pd.DataFrame(
        {'movie': movies['title'], 
         'similarity': np.array(similarities[movie_index, :].todense()).squeeze(),
         'vote_count': movies['vote_count']
        })
    
    # Get the top 10 movies with > 10 votes
    top_movies = sim_df[sim_df['vote_count'] > vote_threshold].sort_values(by='similarity', ascending=False).head(10)
    
    return top_movies

In [ ]:
# Test the recommender
similar_movies = content_recommender("Thor", movie_df, similarities, vote_threshold=1000)
similar_movies.head(10)

## Recommendation based on Ingredients

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
pd.set_option('display.max_colwidth', None)

In [ ]:
df_raw = pd.read_csv('Capstone/RAW_recipes.csv')
df_raw.shape

In [ ]:
df_r = df_raw.drop_duplicates('name')
df_r.shape

In [ ]:
df_r = df_r.dropna()
df_r.shape

In [ ]:
df_r2 = df_r.reset_index(drop=True)
df_r2['ingredients'] = df_r2['ingredients'].str.replace('[^\w\s]','')

In [ ]:
df_s = df_r2[:40000]

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df_s['ingredients'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix)

In [ ]:
ingredient_name = 'pepper'


In [ ]:
def recommend_recipe(ingredient_name):

  recipe_index = df_s[df_s['name'].str.contains(ingredient_name)].index[0]
  similar_recipes = list(enumerate(cosine_sim[recipe_index]))
  sorted_recipes = sorted(similar_recipes,key=lambda x:x[1],reverse=True)[1:]
  recommended_recipes = []
  for i in range(30):
      recommended_recipes.append(df_s.iloc[sorted_recipes[i][0]]['name'])

In [ ]:
recommend_recipe

## Clustering based on Cuisine

We will use present list for cuisine type and what course of meal it is.

In [47]:
cuisineslist = ['american', 'italian', 'asian', 'mexican', 'southern', 'french', 'southwestern', 'barbecue', 'indian', 'chinese', 'cajun', 'mediterranean', 'greek', 'english', 'spanish', 'thai', 'german', 'moroccan', 'irish', 'japanese', 'cuban','hawaiian', 'swedish', 'hungarian', 'portuguese']

courselist = ['Main Dishes', 'Desserts', 'Side Dishes', 'Lunch and Snacks', 'Appetizers', 'Salads', 'Breakfast and Brunch', 'Breads', 'Soups', 'Beverages', 'Condiments and Sauces', 'Cocktails']


In [53]:
resultdf2[resultdf2['tags'].str.contains('american', na=False)]

,user,count,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,...,Calories (#),Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat,Carbohydrates (PDV),date,rate,rev
1,1533,119,bacon and cheese egg appetizers,31048,27,23302,2002-06-12,bacon 30minutesorless timetomake course mainingredient cuisine preparation occasion northamerican appetizers breakfast lunch eggsdairy pork american canadian southernunitedstates easy european dinnerparty fingerfood holidayevent kidfriendly picnic english cheese eggs stovetop dietary christmas newyears thanksgiving glutenfree californian pacificnorthwest independenceday northeasternunitedstates freeofsomething meat togo equipment presentation,4,using a knife cut the outside of the egg white just as far as the yolk and carefully remove the egg white leaving the yolks intact mash the egg yolks and stir with the mayonnaise bacon vinegar cheese and salt fill the egg white halves with the mixture top with the crumbled bacon,...,723.9,55.0,14.0,26.0,37.0,58.0,29.0,2002-07-28,5,"A tray of these barely made it to the table.\r\nI never new so many people liked stuffed eggs.\r\nI never changed a thing, just tripled the recipe. Very good recipe its a keeper."
4,1533,119,basil garlic butter,32311,15,35193,2002-06-26,15minutesorless timetomake course cuisine preparation occasion northamerican lowprotein 5ingredientsorless verylowcarbs condimentsetc american easy nocook refrigerator dinnerparty spring summer vegetarian freezer dietary lowsodium seasonal pacificnorthwest lowcarb inexpensive lowinsomething equipment numberofservings technique,6,place the butter in a mixing bowl add basil garlic and cheese and whip with an electric mixer until mixture is smooth and evenly blended add salt to taste pack butter into four crocks allow flavors to blend for at least 4 hours before serving can be frozen for up to 3 months,...,220.2,18.0,71.0,25.0,21.0,19.0,6.0,2005-03-30,5,"Hi Ginny, I make this all the time now and have it in the freezer. Its great for the bbq and i love adding it to mashed potatoed.. there is so many uses for it, thanks."
7,1533,119,best vegan ranch dressing,132916,14,140331,2005-08-09,15minutesorless timetomake course cuisine preparation occasion northamerican lowprotein appetizers salads condimentsetc american easy beginnercook dinnerparty vegan vegetarian dips saladdressings dietary inexpensive lowinsomething 3stepsorless,2,whisk all ingredients together and chill before serving add a little more soy milk if you need to thin dressing,...,150.6,9.0,94.0,0.0,0.0,19.0,8.0,2007-09-10,5,"This was wonderful, I doubled the recipe and served it to a bunch of kids. It went very quickly. Thanks Dragon, great recipe. Dancer^"
9,1533,119,black bean spareribs,23717,75,4470,2002-03-28,weeknight timetomake course mainingredient cuisine preparation occasion northamerican appetizers maindish beans pork canadian oven barbecue potluck dinnerparty broil stovetop dietary blackbeans meat togo equipment grilling 4hoursorless,8,cut the spareribs into serving size pieces put them in a large saucepan and cover with cold water bring to a boil reduce heat and cook until the ribs are tender about 40 minutes drain arrange ribs on a cookie sheet or shallow pan while the ribs are cooking whisk together the black bean sauce vermouth honey ketchup ginger garlic chili sauce green onions and 1 3 cup coriander pour over the ribs turn and coat all sides cover and refrigerate for at least 4 hours or overnight bring to room temp before cooking you may now bbq broil or bake the ribs bbq over medium high heat for about 10 minutes turning and basting once if baking have the oven pre heated to 425f they are done when the are browned and glazed garnish with the remaining coriander,...,110.2,12.0,4.0,4.0,5.0,14.0,2.0,2005-05-12,5,"I made these last weekend Bergy, and I used back ribs. I put it together Friday night and we had them saturday. They tasted great. We baked them. Next time its the bbq.\r\nThanks Bergy for 

In [51]:
resultdf2[resultdf2['tags'].str.contains('lunch', na=False)]

,user,count,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,...,Calories (#),Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat,Carbohydrates (PDV),date,rate,rev
1,1533,119,bacon and cheese egg appetizers,31048,27,23302,2002-06-12,bacon 30minutesorless timetomake course mainingredient cuisine preparation occasion northamerican appetizers breakfast lunch eggsdairy pork american canadian southernunitedstates easy european dinnerparty fingerfood holidayevent kidfriendly picnic english cheese eggs stovetop dietary christmas newyears thanksgiving glutenfree californian pacificnorthwest independenceday northeasternunitedstates freeofsomething meat togo equipment presentation,4,using a knife cut the outside of the egg white just as far as the yolk and carefully remove the egg white leaving the yolks intact mash the egg yolks and stir with the mayonnaise bacon vinegar cheese and salt fill the egg white halves with the mixture top with the crumbled bacon,...,723.9,55.0,14.0,26.0,37.0,58.0,29.0,2002-07-28,5,"A tray of these barely made it to the table.\r\nI never new so many people liked stuffed eggs.\r\nI never changed a thing, just tripled the recipe. Very good recipe its a keeper."
32,1533,119,cinnamon roll toast,57549,10,35526,2003-04-01,15minutesorless timetomake course preparation for1or2 healthy breakfast stovetop dietary equipment numberofservings,6,sprinkle cinnamon on plain piece of bread spread it in and around with a knife butter that side of the bread place buttered side down in a moderately hot skillet until lightly toasted turn and leave on other side just long enough to keep it from feeling soggy spread hot toast with glaze made with confectioners sugar water and vanilla,...,72.3,10.0,0.0,5.0,5.0,15.0,0.0,2005-01-21,5,"I thought I died when i tasted these were fantastic.\r\nI used whole wheat toast, I would really like to try this on english muffins or crumpets. Thanks so much for a dream breakfast Riff."
121,1535,738,get up go bars,63131,10,21752,2003-05-27,15minutesorless timetomake course preparation occasion forlargegroups breakfast desserts easy beginnercook kosher picnic vegan vegetarian cookiesandbrownies barcookies stovetop dietary brownbag togo camping equipment numberofservings,6,line a 9 x 13 baking dish with foil and coat the foil with nonstick cooking spray in a large saucepan combine the sugar corn syrup and peanut butter over mediumhigh heat bring to a boil and cook for 1 minute stirring constantly add the remaining ingredients mix well then spread evenly into the prepared pan let cool then cut into bars and serve or cover until ready to serve,...,168.9,26.0,5.0,0.0,3.0,11.0,1.0,2005-11-10,5,I really enjoyed these because they are so tasty and so different from my usual breakfast. I bought the Grape Nuts and cranberries on purpose to make them but forgot to check how much corn syrup I had. I only had about 1/2 of what I needed so used honey to make up the difference. My bars were a little softer and stickier that what they probably should have been but they sure tasted good! Thanks Mirj!
123,1535,738,1 2 3 granola,13805,35,21752,2001-11-06,60minutesorless timetomake course mainingredient cuisine preparation occasion northamerican granolaandporridge breakfast lunch american oven easy fingerfood kosher vegetarian grains stovetop dietary gifts lowsodium oamcfreezermakeahead lowinsomething pastariceandgrains equipment numberofservings presentation,7,preheat the oven to 350 degrees in a large mixing bowl combine the first 4 ingredients in a 2quart saucepan heat the honey oil and vanilla until syrupy add to the dry ingredients and mix well spread on a jellyroll pan bake for 20 minutes stirring every 5 minutes store in a covered container,...,217.6,13.0,83.0,7.0,5.0,26.0,10.0,2005-08-30,4,"A good basic granola that is easy to dress up with fresh or dried fruit. It is a bit sweet, even the kids thought so, so next time I'll cut back on the honey some. I used butter instead of oil and 